#Text classification

Let's learn how to finetune the pre-trained BERT for text classification tasks. Say, we are performing sentiment analysis. In the sentiment analysis, our goal is to classify whether a sentence is positive or negative. Suppose, we have a dataset containing sentences along with their labels.

Consider a sentence: 'I love Paris'. 
- First, we tokenize the sentence, add the `[CLS]` token at the beginning, and `[SEP]` token at the end of the sentence. 
- Then, we feed the tokens as an input to the pre-trained BERT and get the embeddings of all the tokens.

Next, we ignore the embedding of all other tokens and take only the embedding of `[CLS]` token which is $R_{[CLS]}$. The embedding of the [CLS] token will hold the aggregate representation of the sentence. We feed $R_{[CLS]}$ to a classifier (feed-forward network with softmax function) and train the classifier to perform sentiment analysis.

How does it differ from what we saw at the beginning of the section. How finetuning the pre-trained BERT differs from using the pre-trained BERT as a feature extractor?

- In "Extracting embeddings from pre-trained BERT" section, we learned that after extracting the embedding $R_{[CLS]}$ of a sentence, we feed the $R_{[CLS]}$ to a classifier and train the classifier to perform classification. Similarly, during finetuning, we feed the embedding of $R_{[CLS]}$ to a classifier and train the classifier to perform classification.

The difference is that :
- when we finetune the pre-trained BERT, we can update the weights of the pre-trained BERT along with a classifier. 
- But when we use the pre-trained BERT as a feature extractor, we can update only the weights of a classifier and not the pre-trained BERT.

During finetuning, we can adjust the weights of the model in the following two ways:

- Update the weights of the pre-trained BERT along with the classification layer
- Update only the weights of the classification layer and not the pre-trained BERT. When we do this, it becomes the same as using the pre-trained BERT as a feature extractor

we feed the tokens to the pre-trained BERT and get the embedding of all the tokens. We take the embedding of `[CLS]` token and feed it to a feedforward network with a softmax function and perform classification.

#Finetuning BERT for sentiment analysis

Let's explore how to finetune the pre-trained BERT for a sentiment analysis task with the IMDB dataset. The IMDB dataset consists of movie reviews along with the respective sentiment.

In [1]:
!pip install nlp
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset
import torch
import numpy as np

In [3]:
!gdown https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
dataset = load_dataset('csv',
                       data_files = './imdbs.csv',
                       split = 'train')

Downloading...
From: https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
To: /content/imdbs.csv
100% 132k/132k [00:00<00:00, 105MB/s]


In [4]:
type(dataset)

nlp.arrow_dataset.Dataset

In [5]:
dataset = dataset.train_test_split(test_size=0.3)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
dataset

{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 70),
 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 30)}

In [7]:
train_set = dataset['train']
test_set = dataset['test']

Next, let's download and load the pre-trained BERT model. In this example, we use the pre-trained **bert-base-uncased model**. As we can observe below, since we are performing **sequence classification**, we use the `BertForSequenceClassification` class:

In [8]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Next, we download and load the **tokenizer** which is used for pretraining the bert-base-uncased model. 

As we can observe, we create the tokenizer using the **BertTokenizerFastclass** instead of BertTokenizer. The BertTokenizerFast class has many advantages compared to BertTokenizer.

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Preprocess the dataset

We can preprocess the dataset in a quicker way using our tokenizer. 


For example, consider the sentence: 

- 'I love Paris'.


1. First, we tokenize the sentence and add the `[CLS]` token at the beginning and `[SEP]` token at the end as shown below:

tokens = [ [CLS], I, love, Paris, [SEP] ]

2. Next, we map the tokens to the unique input ids (token ids). Suppose the following are the unique input ids (token ids):

input_ids = [101, 1045, 2293, 3000, 102]

3. Then, we need to add the segment ids (token type ids). Wait, what are segment ids? Suppose we have two sentences in the input. In that case, segment ids are used to distinguish one sentence from the other. All the tokens from the first sentence will be mapped to 0 and all the tokens from the second sentence will be mapped to 1. Since here we have only one sentence, all the tokens will be mapped to 0 as shown below:

token_type_ids = [0, 0, 0, 0, 0]

4. Now, we need to create the attention mask. We know that an attention mask is used to differentiate the actual tokens and `[PAD]` tokens. It will map all the actual tokens to 1 and the `[PAD]` tokens to 0. Suppose, our tokens length should be 5. Now, our tokens list has already 5 tokens. So, we don't have to add `[PAD]` token. Then our attention mask will become:

attention_mask = [1, 1, 1, 1, 1]

- That's it. But instead of doing all the above steps manually, our tokenizer will do these steps for us. We just need to pass the sentence to the tokenizer as shown below:

In [10]:
tokenizer('I love Paris')

{'input_ids': [101, 1045, 2293, 3000, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

With the tokenizer, we can also pass any number of sentences and perform padding dynamically. To do that, we need to set padding to True and also the maximum sequence length. For instance, as shown below, we pass three sentences and we set the maximum sequence length, max_length to 5:

In [11]:
tokenizer(['I love Paris',
           'birds fly',
           'snow fall'],
          padding = True, max_length=5)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2357: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'input_ids': [[101, 1045, 2293, 3000, 102], [101, 5055, 4875, 102, 0], [101, 4586, 2991, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 1, 0]]}

That's it, with the tokenizer, we can easily preprocess our dataset. So we define a function called preprocess for processing the dataset as shown below:

In [12]:
def preprocess(data):
    return tokenizer(data['text'], padding=True, truncation=True)

Now, we preprocess the train and test set using the preprocess function:

In [13]:
#!pip install dill==0.3.4

In [14]:
train_set = train_set.map(preprocess, batched=True, batch_size=len(train_set))
test_set = test_set.map(preprocess, batched=True, batch_size=len(test_set))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Next, we use the `set_format` function and select the columns which we need in our dataset and also in which format we need them as shown below:

In [15]:
train_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

That's it. Now that we have the dataset ready, let's train the model.

# Training the Model

Define the batch size and epoch size:

In [16]:
batch_size = 8
epochs = 2

Define the warmup steps and weight decay:

In [17]:
warmup_steps = 500
weight_decay = 0.01

Define the training arguments:

In [20]:
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    logging_dir='./logs',
)

Now define the trainer:

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set
)

Start training the model:

In [22]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 70
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18
  Number of trainable parameters = 109483778


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18, training_loss=0.6730284690856934, metrics={'train_runtime': 15.7706, 'train_samples_per_second': 8.877, 'train_steps_per_second': 1.141, 'total_flos': 36835547750400.0, 'train_loss': 0.6730284690856934, 'epoch': 2.0})

In [23]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


{'eval_loss': 0.6668393611907959,
 'eval_runtime': 0.9766,
 'eval_samples_per_second': 30.718,
 'eval_steps_per_second': 4.096,
 'epoch': 2.0}